In [ ]:
!git clone https://github.com/dvolchek/RNN_model_numpy.git
import shutil
shutil.copyfile('/content/RNN_model_numpy/rnn/RNN_utils.py', 'RNN_utils.py')
shutil.copyfile('/content/RNN_model_numpy/rnn/RNN.py', 'RNN.py')
shutil.copyfile('/content/RNN_model_numpy/rnn/names.txt', 'names.txt')

Cloning into 'RNN_model_numpy'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 169 (delta 57), reused 129 (delta 36), pack-reused 0
Receiving objects: 100% (169/169), 3.16 MiB | 7.04 MiB/s, done.
Resolving deltas: 100% (57/57), done.


'names.txt'

### RNN example

In this notebook we will see an example of the RNN built in `RNN.py`. In this example we will be using the data `person_names.txt` to create new names.

In [ ]:
import numpy as np
from RNN import RNNModel
from RNN_utils import SGD, one_hot_encoding, Tanh, Softmax, CrossEntropyLoss

Lets set the random.seed in order to generate always the same weights.

In [ ]:
np.random.seed(1)

### Data
The data contains 18239 names.

In [ ]:
person_names = open('names.txt', 'r').read()
person_names= person_names.lower()
characters = list(set(person_names))

character_to_index = {character:index for index,character in enumerate(sorted(characters))}
index_to_character = {index:character for index,character in enumerate(sorted(characters))}

with open("names.txt") as f:
    person_names = f.readlines()

person_names = [name.lower().strip() for name in person_names]
np.random.shuffle(person_names)

Example of some of the names contained in person_names.txt

In [ ]:
print(person_names[:5])

['ракип', 'айро', 'катаюн', 'микаиль', 'жанриэта']


### Model

#### Define the model

In [ ]:
alphabet = len(character_to_index.keys())

num_epochs = 40000
input_dim = alphabet
output_dim = alphabet
hidden_dim = 100

# initialize and define the model hyperparamaters
model = RNNModel(input_dim, output_dim, hidden_dim)
optim = SGD(lr=0.001)
costs = []

#### Train the model

In [ ]:
from tqdm.notebook import tqdm
# Training
for epoch in tqdm(range(num_epochs+1)):

    # create the X inputs and Y labels
    index = epoch % len(person_names)
    X = [None] + [character_to_index[ch] for ch in person_names[index]]
    Y = X[1:] + [character_to_index["\n"]]

    # transform the input X and label Y into one hot enconding.
    X = one_hot_encoding(X, input_dim)
    Y = one_hot_encoding(Y, output_dim)

    # steps of the model
    model.forward(X)
    cost = model.loss(Y)
    model.backward()
    # clip gradients
    model.clip(clip_value=1)
    # optimize
    model.optimize(optim)

    if epoch % 10000 == 0:
        print ("Loss после эпохи %d: %f" % (epoch, cost))
        costs.append(cost)

        print('Сгенерированные имена:', '\n')
        names = [model.generate_names(index_to_character).replace("\n", "") for i in range(4)]
        print('  |  '.join(names))
        print('--------------------------------------')

  0%|          | 0/40001 [00:00<?, ?it/s]

Loss после эпохи 0: 21.732622
Сгенерированные имена: 

хрйблогйчц  |    |  бйчрятщрзкьшфка  |  кьагьоыбкжуенцх
--------------------------------------
Loss после эпохи 10000: 34.402738
Сгенерированные имена: 

сзуйия  |  блонелд  |  фарирада  |  идан
--------------------------------------
Loss после эпохи 20000: 15.229887
Сгенерированные имена: 

осламинлй  |  асмал  |  эуман  |  сакндон
--------------------------------------
Loss после эпохи 30000: 15.725292
Сгенерированные имена: 

жакомала  |  дакминма  |  илуна  |  влидория
--------------------------------------
Loss после эпохи 40000: 10.700924
Сгенерированные имена: 

агуфаам  |  тжанческа  |  аэврма  |  гиланат
--------------------------------------
